In [15]:
!rm -rf /kaggle/working/
!git clone https://github.com/svenspa/FML.git

In [16]:
import os
origin_path = "/kaggle/working"
fml_path = "/kaggle/working/FML"

In [17]:
!pip install arch
!pip install yfinance
!pip install fastparquet

In [18]:
os.chdir(fml_path)

In [19]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import yfinance as yf

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

from data import DataGJR, DataRes
from market_dynamics import bs_call_price
from models import ControlNet, EnsembleNet
from train import train_val
from risk_measures import median_loss
from utils import call_payoff, stochastic_integral


In [20]:
GJR_FOLDER = "../input/gjr-vol/v2_2/v2/"
RES_FOLDER = "../input/gjr-vol/ressim/resssim/"

In [21]:
os.chdir(origin_path)

## Data

In [22]:
ticker = yf.Ticker('^GSPC')
hist = ticker.history(start='2007-06-11', end='2012-04-10').Close
plt.plot(hist);

In [23]:
# From fitting the GARCH model using R get following parameters

SIGMA = 0.1221684
MU = 0.056105

SQRT_252 = 252 ** 0.5

In [55]:
h_params = {"N_SIM": 4000,
            "RF": 0,
            "N_DIMS": 2,
            "FC_DIM": 20,
            "LR": 0.005,
            "GAMMA": 0.96,
            "EPOCHS": 10,
            "BATCH_SIZE": 256,}

In [56]:
initial_value = hist.iloc[0]
strike = initial_value
n_steps = 30
price = bs_call_price(n_steps, initial_value, SIGMA, h_params["RF"], strike)
initial_value, strike, SIGMA, price

In [57]:
dt = DataGJR(GJR_FOLDER, price, call_payoff, {"strike": strike}, splits=h_params["N_SIM"], S0=initial_value, sigma_0=SIGMA, mu_const=MU, take_log=True, vol_feature=False)

## Training

In [58]:
%%time

# Training three models on different training paths

n_models = 3
models, optimizers, schedulers = [], [], []
n = 1000
val_n = 20
criterion = torch.nn.MSELoss()

for _ in range(n_models):
    model = ControlNet(n_steps, 1, [20, 20], 1, learn_price=False, learn_vol=False)
    models.append(model)
    optimizer = Adam(model.parameters(), lr=h_params["LR"])
    optimizers.append(optimizer)
    schedulers.append(ExponentialLR(optimizer, gamma=h_params["GAMMA"]))

idx_list = np.array_split(np.arange(n) + 1, 3)
val_idx_list = [np.arange(start=idx[-1]+1, stop=idx[-1]+1+val_n) for idx in idx_list]

# Training each model

res_list = []
for model, optimizer, scheduler, indices, val_indices in zip(models, optimizers, schedulers, idx_list, val_idx_list):
    results = train_val(dt, model, criterion, optimizer, h_params["EPOCHS"], indices, val_indices, scheduler, metric=median_loss, val_every=5)
    res_list.append(results)

ens = EnsembleNet(models)

## Performance on GJR validation paths & Delta Hedging Benchmark

In [59]:
# Cython code based on: https://github.com/JackJacquier/python-for-finance/blob/master/Session-7-Cython/Intro_to_Cython.ipynb

In [60]:
%load_ext Cython

In [61]:
%%cython --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
import numpy as np
cimport numpy as cnp

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef cnp.ndarray[cnp.double_t, ndim=1] cython_bs_delta(cnp.ndarray[cnp.double_t, ndim=1] sigma, cnp.ndarray[cnp.double_t, ndim=1] S0,double K, cnp.ndarray[cnp.double_t, ndim=1] T):
    cdef int size=len(S0)
    cdef double k, d1, sigmaT
    cdef cnp.ndarray[cnp.double_t, ndim=1] result=np.zeros(size)
        
    with nogil:
        for i in range(size):
            sigmaT = sigma[i] * sqrt(T[i])
            k = log(S0[i] / K)
            d1 = k / sigmaT + 0.5*sigmaT
            result[i] = gaussian_cdf(d1)
        
    return result

In [62]:
# make index to pass decreasing time to maturity to the bs_delta function
idx = np.arange(n_steps) + 1
idx = idx[::-1]

In [63]:
def cython_d_hedge(x, vol):
    return cython_bs_delta(vol.numpy().astype(np.float64), torch.exp(x).numpy().astype(np.float64), strike, idx.astype(np.float64) *  ( 1 / 365))

def hedge_diff(x, x_inc, payoff, price, vol, model=ens, normalized=True, x1=None):

    # calculate delta hedge; assuming x is log price!
    if normalized:
        path = initial_value * x
    else:
        path = x
    
    d_hedge_list = [cython_d_hedge(a.squeeze(), b.squeeze()) for a, b in zip(path, vol)]
    si_delta = stochastic_integral(x_inc, torch.Tensor(d_hedge_list))
    diff_delta = (price.squeeze() + si_delta).float() - payoff.float()

    # calculating model hedge
    if dt.vol_feature:
        output = model(x, x1)
    else:
        output = model(x)

    if model.learn_price:
        output, price = output

    si = stochastic_integral(x_inc, output)
    diff = (price.squeeze() + si).float() - payoff.float()

    return diff, diff_delta

In [64]:
diffs, diffs_delta = [], []

for i in np.arange(start=3500, stop=3700):
    
    if dt.vol_feature:
        x, x1, x_inc, payoff, price = dt[i]
    else:
        x, x_inc, payoff, price = dt[i]
        x1 = None
        
    vol = dt.get_vol(i)
    diff, diff_delta = hedge_diff(x, x_inc, payoff, price, vol, x1=x1)
    diffs.append(diff)
    diffs_delta.append(diff_delta)
    
d = torch.cat(diffs)
d_delta = torch.cat(diffs_delta)

In [65]:
print('Model:')
print(f"RMSE {(d ** 2).mean() ** 0.5}")
print(f"Using median {(d ** 2).median() ** 0.5}")
print(f"Using mode {(d ** 2).mode()[0] ** 0.5}")

print('Delta Hedge:')
print(f"RMSE {(d_delta ** 2).mean() ** 0.5}")
print(f"Using median {(d_delta ** 2).median() ** 0.5}")
print(f"Using mode {(d_delta ** 2).mode()[0] ** 0.5}")

In [66]:
plt.hist([d.detach().numpy(), d_delta.detach().numpy()], range=(-250, 100), bins=50, label=['Model', 'Delta'])
plt.legend();

In [67]:
pd.Series(d_delta.detach()).describe()

In [68]:
d_series = pd.Series(d.detach())
d_series.describe()

## Performance on real paths

In [75]:
# Creating the validation paths
n_steps = 30# Creating the validation paths

n_steps = 30
step = 5  # number of days by which to shift the window
sigma_est_period = 30
valid_paths, valid_prices, valid_strikes, sigma_estimates = [], [], [], []

for i in np.arange(start=sigma_est_period, stop=len(hist) - n_steps, step=step):
    
    valid_path = hist[i:i+n_steps+1]
    simga_est_path = hist[i-sigma_est_period:i]
    sigma_est = simga_est_path.diff().dropna().std() / 100
    sigma_estimates.append(sigma_est)

    S0 = valid_path[0]
    valid_price = bs_call_price(n_steps, S0, sigma=sigma_est, rf=0, strike=S0)      

    valid_paths.append(valid_path)
    valid_strikes.append(S0)
    valid_prices.append(valid_price)

valid_paths = np.array(valid_paths)
valid_paths = valid_paths.reshape(valid_paths.shape[0], valid_paths.shape[1], 1)
valid_paths = torch.from_numpy(valid_paths).float()

valid_prices = torch.Tensor(valid_prices)
valid_strikes = torch.Tensor(valid_strikes)

In [76]:
if dt.take_log:
    paths = valid_paths.squeeze().T /  valid_paths[:, 0].squeeze()
    paths = paths.T
    paths = torch.unsqueeze(paths, dim=2)
    paths = torch.log(paths[:, :-1])
    x_inc = valid_paths.squeeze().diff()

    payoff = call_payoff(valid_paths.squeeze(), strike=valid_strikes)

In [77]:
output = ens(paths)
si = stochastic_integral(x_inc, output)
diff = (valid_prices.squeeze() + si).float() - payoff.float()
d = diff

In [78]:
print('Model:')
print(f"RMSE {(d ** 2).mean() ** 0.5}")
print(f"Using median {(d ** 2).median() ** 0.5}")

In [79]:
plt.hist(d.detach().numpy(), range=(-250, 100), bins=50, label=['Model'])
plt.legend();

In [80]:
d_series = pd.Series(d.detach())
d_series.describe()

## Reservoir paths

In [81]:
res_dt = DataRes(RES_FOLDER, price, call_payoff, {"strike": strike}, splits=100, S0=initial_value, take_log=True)

In [82]:
diffs = []

for i in range(100):
    x, x_inc, payoff, price = res_dt[i]
    output = ens(x)
    si = stochastic_integral(x_inc, output)
    diff = (price.squeeze() + si).float() - payoff.float()
    diffs.append(diff)
    
d = torch.cat(diffs)

print('Model:')
print(f"RMSE {(d ** 2).mean() ** 0.5}")
print(f"Using median {(d ** 2).median() ** 0.5}")
print(f"Using mode {(d ** 2).mode()[0] ** 0.5}")

In [83]:
plt.hist(d.detach().numpy(), range=(-250, 100), bins=50, label=['Model'])
plt.legend();

## Compare with training on reservoir paths

In [84]:
%%time

# Training three models on different training paths

n_models = 3
models, optimizers, schedulers = [], [], []
n = 900
val_n = 20
criterion = torch.nn.MSELoss()

for _ in range(n_models):
    model = ControlNet(n_steps, 1, [20, 20], 1, learn_price=False, learn_vol=False)
    models.append(model)
    optimizer = Adam(model.parameters(), lr=h_params["LR"])
    optimizers.append(optimizer)
    schedulers.append(ExponentialLR(optimizer, gamma=h_params["GAMMA"]))

idx_list = np.array_split(np.arange(n) + 1, 3)
val_idx_list = [np.arange(start=idx[-1]+1, stop=idx[-1]+1+val_n) for idx in idx_list]

# Training each model

res_list = []
for model, optimizer, scheduler, indices, val_indices in zip(models, optimizers, schedulers, idx_list, val_idx_list):
    results = train_val(res_dt, model, criterion, optimizer, h_params["EPOCHS"], indices, val_indices, scheduler, metric=median_loss, val_every=5)
    res_list.append(results)

ens = EnsembleNet(models)

In [85]:
# Out of sample performance

diffs = []

for i in np.arange(n+10,1000):
    x, x_inc, payoff, price = res_dt[i]
    output = ens(x)
    si = stochastic_integral(x_inc, output)
    diff = (price.squeeze() + si).float() - payoff.float()
    diffs.append(diff)
    
d = torch.cat(diffs)

print('Model:')
print(f"RMSE {(d ** 2).mean() ** 0.5}")
print(f"Using median {(d ** 2).median() ** 0.5}")
print(f"Using mode {(d ** 2).mode()[0] ** 0.5}")

In [86]:
plt.hist(d.detach().numpy(), range=(-250, 100), bins=50, label=['Model'])
plt.legend();

In [87]:
# Performance on real data

if dt.take_log:
    paths = valid_paths.squeeze().T /  valid_paths[:, 0].squeeze()
    paths = paths.T
    paths = torch.unsqueeze(paths, dim=2)
    paths = torch.log(paths[:, :-1])
    x_inc = valid_paths.squeeze().diff()
    payoff = call_payoff(valid_paths.squeeze(), strike=valid_strikes)
    
    
output = ens(paths)
si = stochastic_integral(x_inc, output)
diff = (valid_prices.squeeze() + si).float() - payoff.float()
d = diff

print('Model:')
print(f"RMSE {(d ** 2).mean() ** 0.5}")
print(f"Using median {(d ** 2).median() ** 0.5}")
print(f"Using mode {(d ** 2).mode()[0] ** 0.5}")

In [88]:
plt.hist(d.detach().numpy(), range=(-250, 100), bins=50, label=['Model'])
plt.legend();

### Plotting training, validation loss & metric

In [89]:
l, vl, m = res_list[1]
plt.plot(l, marker='o')

In [90]:
plt.plot(vl, marker='o')

In [91]:
gjr_x, _, _, _, = dt[1]
res_x, _, _, _, = res_dt[1]
gjr_x, res_x = torch.exp(gjr_x), torch.exp(res_x)

gjr_example_paths = gjr_x[:10]
res_example_paths= res_x[:10]

fut = ticker.history(start='2018-01-02', end='2019-01-02').Close
m = np.floor(len(fut) / n_steps)
real_example_paths = []

for i in np.arange(start=0, stop=(m-1)*n_steps, step=n_steps):
    i = int(i)
    path = fut.iloc[i:i+n_steps].values
    path = path / path[0]
    real_example_paths.append(path)

plt.plot(gjr_example_paths.T.squeeze(), c='blue');
plt.plot(res_example_paths.T.squeeze(), c='orange');
plt.plot(np.array(real_example_paths).T, c='black');

# plt.legend(['GJR-GARCH', 'Reservoir Computing', 'Real paths'])
gjrpath_leg = Line2D([0], [0], color='blue', label='GJR-GARCH')
rc_leg = Line2D([0], [0], color='orange', label='Reservoir Computing')
real_leg = Line2D([0], [0], color='black', label='Real paths')
plt.legend(handles=[gjrpath_leg, rc_leg, real_leg])

plt.savefig('compare_paths.eps', format='eps')

plt.show()